# Importar bibliotecas

In [69]:
import pandas as pd
import numpy as np
from numpy import int64
import time
from datetime import datetime

#!pip install elasticsearch
from elasticsearch import Elasticsearch
import elasticsearch.helpers

import warnings
warnings.filterwarnings("ignore")

# Baixar base de dados

In [70]:
user = 'XXXXXX'
pwd = 'XXXXXX'
index = 'XXXXX'

# Define a URL com usuario e senha
url = 'https://' + user + ':' + pwd + '@elasticsearch-saps.saude.gov.br'

# Faz uma requisição POST com a URL definida
es = Elasticsearch([url], send_get_body_as='POST')

# Verifica toda a base de dados disponível
body={"query": {"match_all": {}}}

# Cria um documento contendo a base de dados definida pelo index
results = elasticsearch.helpers.scan(es, query=body, index=index)

# Cria um Data Frame chamado df, com toda a base de dados de SRAG do DF
df = pd.DataFrame.from_dict([document['_source'] for document in results])

# 1 - Tratar os Dados

# 1.1 - Transformar colunas em maiúsculo e alterar ordem das colunas

In [71]:
# coloca o nome das colunas em maiúsculo
df.columns = df.columns.str.upper()

# Cria uma lista com o nome das colunas na ordem solicitada
ordem_colunas = ['NU_NOTIFIC', 'DT_NOTIFIC', 'SEM_NOT', 'DT_SIN_PRI', 'SEM_PRI', 'SG_UF_NOT', 'ID_REGIONA', 'CO_REGIONA', 'ID_MUNICIP', 'CO_MUN_NOT', 'ID_UNIDADE',
 'CO_UNI_NOT', 'NU_CPF', 'NM_PACIENT', 'CS_SEXO', 'DT_NASC', 'NU_IDADE_N', 'TP_IDADE', 'COD_IDADE', 'CS_GESTANT', 'CS_RACA', 'CS_ETINIA', 'CS_ESCOL_N', 'NM_MAE_PAC',
 'NU_CEP', 'ID_PAIS', 'CO_PAIS', 'SG_UF', 'ID_RG_RESI', 'CO_RG_RESI', 'ID_MN_RESI', 'CO_MUN_RES', 'NM_BAIRRO', 'NM_LOGRADO', 'NU_NUMERO', 'NM_COMPLEM', 'NU_DDD_TEL',
 'NU_TELEFON', 'CS_ZONA', 'SURTO_SG', 'NOSOCOMIAL', 'AVE_SUINO', 'FEBRE', 'TOSSE', 'GARGANTA', 'DISPNEIA', 'DESC_RESP', 'SATURACAO', 'DIARREIA', 'VOMITO', 'OUTRO_SIN',
 'OUTRO_DES', 'FATOR_RISC', 'PUERPERA', 'CARDIOPATI', 'HEMATOLOGI', 'SIND_DOWN', 'HEPATICA', 'ASMA', 'DIABETES', 'NEUROLOGIC', 'PNEUMOPATI', 'IMUNODEPRE', 'RENAL',
 'OBESIDADE', 'OBES_IMC', 'OUT_MORBI', 'MORB_DESC', 'VACINA', 'DT_UT_DOSE', 'MAE_VAC', 'DT_VAC_MAE', 'M_AMAMENTA', 'DT_DOSEUNI', 'DT_1_DOSE', 'DT_2_DOSE', 'ANTIVIRAL',
 'TP_ANTIVIR', 'OUT_ANTIV', 'DT_ANTIVIR', 'HOSPITAL', 'DT_INTERNA', 'SG_UF_INTE', 'ID_RG_INTE', 'CO_RG_INTE', 'ID_MN_INTE', 'CO_MU_INTE', 'NM_UN_INTE', 'CO_UN_INTE',
 'UTI', 'DT_ENTUTI', 'DT_SAIDUTI', 'SUPORT_VEN', 'RAIOX_RES', 'RAIOX_OUT', 'DT_RAIOX', 'AMOSTRA', 'DT_COLETA', 'TP_AMOSTRA', 'OUT_AMOST', 'REQUI_GAL', 'PCR_RESUL',
 'DT_PCR', 'POS_PCRFLU', 'TP_FLU_PCR', 'PCR_FLUASU', 'FLUASU_OUT', 'PCR_FLUBLI', 'FLUBLI_OUT', 'POS_PCROUT', 'PCR_VSR', 'PCR_PARA1', 'PCR_PARA2', 'PCR_PARA3', 'PCR_PARA4',
 'PCR_ADENO', 'PCR_METAP', 'PCR_BOCA', 'PCR_RINO', 'PCR_OUTRO', 'DS_PCR_OUT', 'LAB_PCR', 'CO_LAB_PCR', 'CLASSI_FIN', 'CLASSI_OUT', 'CRITERIO', 'EVOLUCAO', 'DT_EVOLUCA',
 'DT_ENCERRA', 'OBSERVA', 'NOME_PROF', 'REG_PROF', 'DT_DIGITA', 'HISTO_VGM', 'PAIS_VGM', 'CO_PS_VGM', 'LO_PS_VGM', 'DT_VGM', 'DT_RT_VGM', 'PCR_SARS2', 'PAC_COCBO',
 'PAC_DSCBO', 'OUT_ANIM', 'DOR_ABD', 'FADIGA', 'PERD_OLFT', 'PERD_PALA', 'TOMO_RES', 'TOMO_OUT', 'DT_TOMO', 'TP_TES_AN', 'DT_RES_AN', 'RES_AN', 'LAB_AN', 'CO_LAB_AN',
 'POS_AN_FLU', 'TP_FLU_AN', 'POS_AN_OUT', 'AN_SARS2', 'AN_VSR', 'AN_PARA1', 'AN_PARA2', 'AN_PARA3', 'AN_ADENO', 'AN_OUTRO', 'DS_AN_OUT', 'TP_AM_SOR', 'SOR_OUT', 'DT_CO_SOR',
 'TP_SOR', 'OUT_SOR', 'DT_RES', 'RES_IGG', 'RES_IGM', 'RES_IGA', 'NU_DO', 'POV_CT', 'TP_POV_CT', 'TEM_CPF', 'ESTRANG', 'NU_CNS', 'VACINA_COV', 'DOSE_1_COV', 'DOSE_2_COV',
 'LAB_PR_COV', 'LOTE_1_COV', 'LOTE_2_COV', 'FNT_IN_COV']
 
# Deixa as colunas na Ordem  da lista ordem_colunas
df = df[ordem_colunas]

# 1.2 -  Alterar os tipos de dados

In [72]:
# Cria uma lista com as colunas que devem ser do tipo datetime
datas = [
'DT_NASC',
'DT_DIGITA',
'DT_SIN_PRI',
'DT_INTERNA',
'DT_ENTUTI',
'DT_SAIDUTI',
'DT_PCR',
'DT_EVOLUCA',
'DT_ENCERRA',
'DT_COLETA',
'DT_NOTIFIC'
]

In [73]:
# Cria uma lista com as colunas que devem ser do tipo float
float_columns = [
 'CO_REGIONA',
 'CS_ESCOL_N',
 'NU_CEP',
 'CO_RG_RESI',
 'NU_DDD_TEL',
 'CS_ZONA',
 'SURTO_SG',
 'NOSOCOMIAL',
 'AVE_SUINO',
 'FEBRE',
 'TOSSE',
 'GARGANTA',
 'DISPNEIA',
 'DESC_RESP',
 'SATURACAO',
 'DIARREIA',
 'VOMITO',
 'OUTRO_SIN',
 'PUERPERA',
 'CARDIOPATI',
 'HEMATOLOGI',
 'SIND_DOWN',
 'HEPATICA',
 'ASMA',
 'DIABETES',
 'NEUROLOGIC',
 'PNEUMOPATI',
 'IMUNODEPRE',
 'RENAL',
 'OBESIDADE',
 'OBES_IMC',
 'OUT_MORBI',
 'VACINA',
 'MAE_VAC',
 'M_AMAMENTA',
 'DT_DOSEUNI',
 'DT_1_DOSE',
 'DT_2_DOSE',
 'ANTIVIRAL',
 'TP_ANTIVIR',
 'HOSPITAL',
 'CO_RG_INTE',
 'CO_MU_INTE',
 'CO_UN_INTE',
 'UTI',
 'SUPORT_VEN',
 'RAIOX_RES',
 'AMOSTRA',
 'TP_AMOSTRA',
 'REQUI_GAL',
 'PCR_RESUL',
 'POS_PCRFLU',
 'TP_FLU_PCR',
 'PCR_FLUASU',
 'FLUASU_OUT',
 'PCR_FLUBLI',
 'FLUBLI_OUT',
 'POS_PCROUT',
 'PCR_VSR',
 'PCR_PARA1',
 'PCR_PARA2',
 'PCR_PARA3',
 'PCR_PARA4',
 'PCR_ADENO',
 'PCR_METAP',
 'PCR_BOCA',
 'PCR_RINO',
 'PCR_OUTRO',
 'CO_LAB_PCR',
 'CLASSI_FIN',
 'CRITERIO',
 'EVOLUCAO',
 'PAIS_VGM',
 'CO_PS_VGM',
 'LO_PS_VGM',
 'DT_VGM',
 'DT_RT_VGM',
 'PCR_SARS2',
 'DOR_ABD',
 'FADIGA',
 'PERD_OLFT',
 'PERD_PALA',
 'RES_AN',
 'CO_LAB_AN',
 'POS_AN_FLU',
 'TP_FLU_AN',
 'POS_AN_OUT',
 'AN_SARS2',
 'AN_VSR',
 'AN_PARA1',
 'AN_PARA2',
 'AN_PARA3',
 'AN_ADENO',
 'AN_OUTRO',
 'RES_IGG',
 'RES_IGM',
 'RES_IGA',
 'POV_CT',
 'TEM_CPF',
 'ESTRANG',
 'NU_CNS',
 'VACINA_COV',
 'FNT_IN_COV',
 'SEM_NOT',
 'SEM_PRI',
 'CO_MUN_NOT',
 'CO_UNI_NOT',
 'NU_IDADE_N',
 'TP_IDADE',
 'COD_IDADE',
 'CS_GESTANT',
 'CS_RACA',
 'CO_PAIS',
 'CO_MUN_RES',
 'FATOR_RISC',
 'HISTO_VGM',
 'TOMO_RES',
 'TP_TES_AN',
 'TP_AM_SOR',
 'TP_SOR',
 'NU_DO'
 ]


In [74]:
# Cria uma lista com as colunas que devem ser do tipo object
object_columns = [
 'NU_NOTIFIC',
 'SG_UF_NOT',
 'ID_REGIONA',
 'ID_MUNICIP',
 'ID_UNIDADE',
 'NU_CPF',
 'NM_PACIENT',
 'CS_SEXO',
 'CS_ETINIA',
 'NM_MAE_PAC',
 'ID_PAIS',
 'SG_UF',
 'ID_RG_RESI',
 'ID_MN_RESI',
 'NM_BAIRRO',
 'NM_LOGRADO',
 'NU_NUMERO',
 'NM_COMPLEM',
 'NU_TELEFON',
 'OUTRO_DES',
 'MORB_DESC',
 'DT_UT_DOSE',
 'DT_VAC_MAE',
 'OUT_ANTIV',
 'DT_ANTIVIR',
 'SG_UF_INTE',
 'ID_RG_INTE',
 'ID_MN_INTE',
 'NM_UN_INTE',
 'RAIOX_OUT',
 'DT_RAIOX',
 'OUT_AMOST',
 'DS_PCR_OUT',
 'LAB_PCR',
 'CLASSI_OUT',
 'OBSERVA',
 'NOME_PROF',
 'REG_PROF',
 'PAC_COCBO',
 'PAC_DSCBO',
 'OUT_ANIM',
 'TOMO_OUT',
 'DT_TOMO',
 'DT_RES_AN',
 'LAB_AN',
 'DS_AN_OUT',
 'SOR_OUT',
 'DT_CO_SOR',
 'OUT_SOR',
 'DT_RES',
 'TP_POV_CT',
 'DOSE_1_COV',
 'DOSE_2_COV',
 'LAB_PR_COV',
 'LOTE_1_COV',
 'LOTE_2_COV']

In [75]:
# Transforma as colunas de data para o tipo datetime64[ns]
df[datas] = df[datas].apply(pd.to_datetime,infer_datetime_format = True, errors='coerce')

# Transforma as colunas de float_columns para o tipo float
df[float_columns] = df[float_columns].apply(pd.to_numeric, errors='coerce')

# Transforma as colunas de object_columns para o tipo object
df[object_columns] = df[object_columns].astype(object)

# 1.3 - Criação e manipulação de variáveis essenciais

In [76]:
# Cria a coluna IDADE_REGISTRO subtraindo a data do dia de registro no sistema pela data de nascimento, converte para ano e arredonda para nenhuma casa decimal
df['IDADE_REGISTRO'] = ((df['DT_DIGITA'] - df['DT_NASC'])/ np.timedelta64(1, 'Y')).round()

# Transforma a coluna 'NU_NOTIFIC' em string
df['NU_NOTIFIC'] = df['NU_NOTIFIC'].apply(str)

# Transforma a coluna de CPF em String e retira o .0 da direita
df['NU_CPF'] = df['NU_CPF'].apply(str).apply(lambda x: x.replace('.0',''))

# Cria uma coluna para reinternação e acrescenta 0 em todas as linhas, posteriormente, o que for identificado como reinternação terá 1
df['REINTERNACAO'] = 0

# Altera os valores que estãoe escritos como None para núlos
df.replace('None', np.nan, inplace=True)

# 1.4 - Criação de Filtro para casos do ano vigente

In [77]:
# Cria uma lista com anos e seus inícios de calendários epidemológicos
data = [[2020, '12/29/2019'], [2021, '01/03/2021'], [2022, '01/02/2022']]
 
# Cria uma planilha com os anos e dias de início
calendario_epidemologico = pd.DataFrame(data, columns = ['ANO', 'INICIO'])

# Converte a coluna de dias para o tipo data
calendario_epidemologico['INICIO'] = calendario_epidemologico['INICIO'].apply(pd.to_datetime,infer_datetime_format = True, errors='coerce')

# Transforma a coluna de Ano em Index
calendario_epidemologico.set_index('ANO', inplace = True)

# Cria uma variável ano com o ano atual
ano = datetime.now().year

# Cria uma variável dia_calendario que recebe o primeiro dia da semana epidemologica do ano atual 
dia_calendario = calendario_epidemologico['INICIO'][ano]

# Deixa na planilha df somente os registros a partir do primeiro dia de semana epidemologica do ano atual
df = df.loc[df['DT_SIN_PRI']>= dia_calendario]

# Reseta o index
df.reset_index(drop = True, inplace = True)

# 2 - Identificação de Registros Duplicados

In [78]:
# Cria um df com valores que se repetem em 'NM_PACIENT' e 'DT_NASC'
df_nome = df[df[['NM_PACIENT','DT_NASC']].duplicated(keep=False)]

# Cria o df df_cpf com os registros onde o cpf não são 'nan
df_cpf = df[df['NU_CPF'].notnull()]
# Deixa somente os registros duplicados
df_cpf = df_cpf[df_cpf[['NU_CPF']].duplicated(keep=False)]

# Junta os valores repetidos dos dois df acima
df_repetidos = pd.concat([df_nome,df_cpf])

# Apaga os registros que possuem valor repetido na coluna 'NU_NOTIFIC', deixando somente uma instância no 'df_repetidos'
df_repetidos = df_repetidos.drop_duplicates(subset=['NU_NOTIFIC'], keep='last')

# Apaga o 'df_nome', 'df_sem_nan_cpf' e o 'df_cpf'
del df_cpf, df_nome        


# 2.1 - Acrescentar descrição em REINTERNACAO
### 0 = não possui repetição de nome nem CPF
### 1 = Duplicidade que será mantida (Primeiro registro)
### 2 = Duplicidade que será apagada (Segundo registro)
### 3 = Reinternação ( Data de Primeiros Sintomas maior que 15 dias )


In [79]:
# Cria um loop que vai percorrer a coluna 'NM_PACIENT' de df_repetidos
for nome in df_repetidos['NM_PACIENT']:

    # Cria a variável linhas e adiciona os valor do número de linahs com nomes iguais
    linhas = df_repetidos.loc[df_repetidos['NM_PACIENT'] == nome].shape[0]
    # Cria uma lista vazia com a quantidade de espaços de linhas 
    ids = [''] * linhas
    # Cria um loop a ser repetido pela quantidade de linhas
    for qtd in range(linhas):
        # Adiciona o index das linhas em ordem na lista ids
        ids[qtd] = df_repetidos.loc[df_repetidos['NM_PACIENT'] == nome]['REINTERNACAO'].index[qtd]
        
            
    # Cria uma variável data com o registro das datas de 'DT_SIN_PRI' dos nomes repetidos 
    data = df_repetidos.loc[df_repetidos['NM_PACIENT'] == nome]['DT_SIN_PRI']

    # Cria uma lista vazia com a quantidade de espaços de linhas 
    dif = [''] * linhas
    # Cria um loop a ser repetido pela quantidade de linhas
    for qtd in range(linhas):
        # Adiciona na lista a diferença de dias entre o primeiro registro e os demais
        dif[qtd] = (data[ids[qtd]] - data[ids[0]]).days 
    # Apaga o primeiro registro da lista dif (é zero por padrão)
    del dif[0]

    # Variável para pegar o índice de ids
    i = 1
    # Irá criar um loop que percorrerá os valores das diferenças em dif
    for diferenca in dif:
        # Verifica se a diferença do dia do primeiro com o segundo registro está entre 0 e 15 
        if diferenca <= 15 and diferenca >= 0:

            # Verifica se o segundo registro possui 0 em REINTERNACAO
            if df_repetidos.loc[(df_repetidos['NU_NOTIFIC'] == df_repetidos['NU_NOTIFIC'][ids[i]]), 'REINTERNACAO'][ids[i]] == 0:
                # Caso seja 0, ele vai receber 2
                df.loc[(df['NU_NOTIFIC'] == df_repetidos['NU_NOTIFIC'][ids[i]]), 'REINTERNACAO'] = 2
                df_repetidos.loc[(df_repetidos['NU_NOTIFIC'] == df_repetidos['NU_NOTIFIC'][ids[i]]), 'REINTERNACAO'] = 2
            
            # Verifica se o primeiro registro possui 0 em REINTERNACAO
            if df_repetidos.loc[(df_repetidos['NU_NOTIFIC'] == df_repetidos['NU_NOTIFIC'][ids[0]]), 'REINTERNACAO'][ids[0]] == 0:
                # Caso seja 0, ele vai receber 1
                df.loc[(df['NU_NOTIFIC'] == df_repetidos['NU_NOTIFIC'][ids[0]]), 'REINTERNACAO'] = 1
                df_repetidos.loc[(df_repetidos['NU_NOTIFIC'] == df_repetidos['NU_NOTIFIC'][ids[0]]), 'REINTERNACAO'] = 1

        # Verifica se a diferença do dia do primeiro com o segundo registro está entre -15 e 0
        elif diferenca >= -15 and diferenca < 0:

            # Verifica se o segundo registro possui 0 em REINTERNACAO
            if df_repetidos.loc[(df_repetidos['NU_NOTIFIC'] == df_repetidos['NU_NOTIFIC'][ids[i]]), 'REINTERNACAO'][ids[i]] == 0:
                # Caso seja 0, ele vai receber 1
                df.loc[(df['NU_NOTIFIC'] == df_repetidos['NU_NOTIFIC'][ids[i]]), 'REINTERNACAO'] = 1
                df_repetidos.loc[(df_repetidos['NU_NOTIFIC'] == df_repetidos['NU_NOTIFIC'][ids[i]]), 'REINTERNACAO'] = 1

            # Verifica se o primeiro registro possui 0 em REINTERNACAO
            if df_repetidos.loc[(df_repetidos['NU_NOTIFIC'] == df_repetidos['NU_NOTIFIC'][ids[0]]), 'REINTERNACAO'][ids[0]] == 0:
                # Caso seja 0, ele vai receber 2
                df.loc[(df['NU_NOTIFIC'] == df_repetidos['NU_NOTIFIC'][ids[0]]), 'REINTERNACAO'] = 2
                df_repetidos.loc[(df_repetidos['NU_NOTIFIC'] == df_repetidos['NU_NOTIFIC'][ids[0]]), 'REINTERNACAO'] = 2

        # Caso a diferença entre o primeiro e o segundo dia seja maior que 15 dias ou menor que -15, executa os códigos abaixo dentro do else
        else:

            # Verifica se o segundo registro possui 0 em REINTERNACAO
            if df_repetidos.loc[(df_repetidos['NU_NOTIFIC'] == df_repetidos['NU_NOTIFIC'][ids[i]]), 'REINTERNACAO'][ids[i]] == 0:
                # Caso seja 0, ele vai receber 3
                df.loc[(df['NU_NOTIFIC'] == df_repetidos['NU_NOTIFIC'][ids[i]]), 'REINTERNACAO'] = 3
                df_repetidos.loc[(df_repetidos['NU_NOTIFIC'] == df_repetidos['NU_NOTIFIC'][ids[i]]), 'REINTERNACAO'] = 3
            
            # Verifica se o primeiro registro possui 0 em REINTERNACAO
            if df_repetidos.loc[(df_repetidos['NU_NOTIFIC'] == df_repetidos['NU_NOTIFIC'][ids[0]]), 'REINTERNACAO'][ids[0]] == 0:
                # Caso seja 0, ele vai receber 3
                df.loc[(df['NU_NOTIFIC'] == df_repetidos['NU_NOTIFIC'][ids[0]]), 'REINTERNACAO'] = 3
                df_repetidos.loc[(df_repetidos['NU_NOTIFIC'] == df_repetidos['NU_NOTIFIC'][ids[0]]), 'REINTERNACAO'] = 3
        i += 1

              
# Apaga os registros em df_repetidos onde o cpf for nulo
df_repetidos = df_repetidos.loc[df_repetidos['NU_CPF'].notnull()]

# Cria um loop que vai percorrer a coluna 'NM_PACIENT' de df_repetidos
for cpf in df_repetidos['NU_CPF']:

    # Cria a variável linhas e adiciona os valor do número de linahs com cpfs iguais
    linhas = df_repetidos.loc[df_repetidos['NU_CPF'] == cpf].shape[0]
    # Cria uma lista vazia com a quantidade de espaços de linhas 
    ids = [''] * linhas
    # Cria um loop a ser repetido pela quantidade de linhas
    for qtd in range(linhas):
        # Adiciona o index das linhas em ordem na lista ids
        ids[qtd] = df_repetidos.loc[df_repetidos['NU_CPF'] == cpf]['REINTERNACAO'].index[qtd]
        
            
    # Cria uma variável data com o registro das datas de 'DT_SIN_PRI' dos cpfs repetidos 
    data = df_repetidos.loc[df_repetidos['NU_CPF'] == cpf]['DT_SIN_PRI']

    # Cria uma lista vazia com a quantidade de espaços de linhas 
    dif = [''] * linhas
    # Cria um loop a ser repetido pela quantidade de linhas
    for qtd in range(linhas):
        # Adiciona na lista a diferença de dias entre o primeiro registro e os demais
        dif[qtd] = (data[ids[qtd]] - data[ids[0]]).days 
    # Apaga o primeiro registro da lista dif (é zero por padrão)
    del dif[0]

    # Variável para pegar o índice de ids
    i = 1
    # Irá criar um loop que percorrerá os valores das diferenças em dif
    for diferenca in dif:
        # Verifica se a diferença do dia do primeiro com o segundo registro está entre 0 e 15 
        if diferenca <= 15 and diferenca >= 0:

            # Verifica se o segundo registro possui 0 em REINTERNACAO
            if df_repetidos.loc[(df_repetidos['NU_NOTIFIC'] == df_repetidos['NU_NOTIFIC'][ids[i]]), 'REINTERNACAO'][ids[i]] == 0:
                # Caso seja 0, ele vai receber 2
                df.loc[(df['NU_NOTIFIC'] == df_repetidos['NU_NOTIFIC'][ids[i]]), 'REINTERNACAO'] = 2
                df_repetidos.loc[(df_repetidos['NU_NOTIFIC'] == df_repetidos['NU_NOTIFIC'][ids[i]]), 'REINTERNACAO'] = 2
            
            # Verifica se o primeiro registro possui 0 em REINTERNACAO
            if df_repetidos.loc[(df_repetidos['NU_NOTIFIC'] == df_repetidos['NU_NOTIFIC'][ids[0]]), 'REINTERNACAO'][ids[0]] == 0:
                # Caso seja 0, ele vai receber 1
                df.loc[(df['NU_NOTIFIC'] == df_repetidos['NU_NOTIFIC'][ids[0]]), 'REINTERNACAO'] = 1
                df_repetidos.loc[(df_repetidos['NU_NOTIFIC'] == df_repetidos['NU_NOTIFIC'][ids[0]]), 'REINTERNACAO'] = 1

        # Verifica se a diferença do dia do primeiro com o segundo registro está entre -15 e 0
        elif diferenca >= -15 and diferenca < 0:

            # Verifica se o segundo registro possui 0 em REINTERNACAO
            if df_repetidos.loc[(df_repetidos['NU_NOTIFIC'] == df_repetidos['NU_NOTIFIC'][ids[i]]), 'REINTERNACAO'][ids[i]] == 0:
                # Caso seja 0, ele vai receber 1
                df.loc[(df['NU_NOTIFIC'] == df_repetidos['NU_NOTIFIC'][ids[i]]), 'REINTERNACAO'] = 1
                df_repetidos.loc[(df_repetidos['NU_NOTIFIC'] == df_repetidos['NU_NOTIFIC'][ids[i]]), 'REINTERNACAO'] = 1

            # Verifica se o primeiro registro possui 0 em REINTERNACAO
            if df_repetidos.loc[(df_repetidos['NU_NOTIFIC'] == df_repetidos['NU_NOTIFIC'][ids[0]]), 'REINTERNACAO'][ids[0]] == 0:
                # Caso seja 0, ele vai receber 2
                df.loc[(df['NU_NOTIFIC'] == df_repetidos['NU_NOTIFIC'][ids[0]]), 'REINTERNACAO'] = 2
                df_repetidos.loc[(df_repetidos['NU_NOTIFIC'] == df_repetidos['NU_NOTIFIC'][ids[0]]), 'REINTERNACAO'] = 2

        # Caso a diferença entre o primeiro e o segundo dia seja maior que 15 dias ou menor que -15, executa os códigos abaixo dentro do else
        else:

            # Verifica se o segundo registro possui 0 em REINTERNACAO
            if df_repetidos.loc[(df_repetidos['NU_NOTIFIC'] == df_repetidos['NU_NOTIFIC'][ids[i]]), 'REINTERNACAO'][ids[i]] == 0:
                # Caso seja 0, ele vai receber 3
                df.loc[(df['NU_NOTIFIC'] == df_repetidos['NU_NOTIFIC'][ids[i]]), 'REINTERNACAO'] = 3
                df_repetidos.loc[(df_repetidos['NU_NOTIFIC'] == df_repetidos['NU_NOTIFIC'][ids[i]]), 'REINTERNACAO'] = 3
            
            # Verifica se o primeiro registro possui 0 em REINTERNACAO
            if df_repetidos.loc[(df_repetidos['NU_NOTIFIC'] == df_repetidos['NU_NOTIFIC'][ids[0]]), 'REINTERNACAO'][ids[0]] == 0:
                # Caso seja 0, ele vai receber 3
                df.loc[(df['NU_NOTIFIC'] == df_repetidos['NU_NOTIFIC'][ids[0]]), 'REINTERNACAO'] = 3
                df_repetidos.loc[(df_repetidos['NU_NOTIFIC'] == df_repetidos['NU_NOTIFIC'][ids[0]]), 'REINTERNACAO'] = 3
        i += 1
    
# Apagas as variáveis usadas no código
del df_repetidos, dif, diferenca, i, ids, linhas, nome,  qtd, data, cpf

# transforma a coluna NU_NOTIFIC em index
df.set_index('NU_NOTIFIC', inplace = True)

# 2.2 - Salva as planilhas com Descrição em Reinternação
Para sonseguir salvar as planilhas, é necessária a criação de uma pasta chamada bases na mesma pasta que o código, e dentro da pasta bases criar uma pasta duplicidades, dentro de duplicidades criar duplicidades por hospitais.

In [80]:
# Cria uma cópia do df antes de converter as datas para gerar as planilhas
dff = df.copy(deep=True)

# Converte as colunas do tipo data que estão no formato americano para o brasileiro
for data in datas:
    df[data] = df[data].dt.strftime('%d/%m/%Y')

del data

In [81]:
# Cria um df com os repetidos apagados 
df_repetidos_apagados = df.loc[df['REINTERNACAO']==2].sort_values(['NM_PACIENT'])
# Salava uma planilha com os repetidos apagados
df_repetidos_apagados.to_excel("bases/duplicidades/duplicidades_apagadas.xlsx")


# Cria uma planilha de apagados por hospitais
df_repetidos_apagados_hospitais = df_repetidos_apagados

# Salva uma planilha com os registros repetidos que não serão apagados
df.loc[df['REINTERNACAO']==1].sort_values(['NM_PACIENT']).to_excel("bases/duplicidades/duplicidades_mantidas.xlsx")

# Todos valores repetidos desconsiderando as reinternações
pd.concat([df.loc[df['REINTERNACAO']==2],df.loc[df['REINTERNACAO']==1]]).drop_duplicates(keep='last').sort_values(['NM_PACIENT']).to_excel("bases/duplicidades/duplicidades_sem_reinternacoes.xlsx")

# Salva uma planilha com todos os valores repetidos, incluindo as reinternações
df.loc[df['REINTERNACAO']!=0].sort_values(['NM_PACIENT']).to_excel("bases/duplicidades/duplicidades_com_reinternacoes.xlsx")

# Salva uma planilha com todos os pacientes de reintenação
df.loc[df['REINTERNACAO']==3].sort_values(['NM_PACIENT']).to_excel("bases/duplicidades/reinternacao.xlsx")

# Deixa no df somente os registros sem duplicidades, salvo os casos de reinternação
df = df.loc[df['REINTERNACAO']!=2]

# Salva uma planilha com 
df.to_excel("bases/planilha_tratada.xlsx")

# 2.3 - Salvar as planilhas de duplicidades de cada hospital

In [82]:
# Define quais colunas irão ser enviadas para os Hospitais - A coluna de NU_NOTIFIC é o index da planilha e por isto não necessita estar definida aqui
colunas_apagados = ['NU_CPF', 'NM_PACIENT', 'DT_NASC', 'ID_UNIDADE', 'DT_SIN_PRI']

# Deixa na planilha df_repetidos_apagados_hospitais somente as colunas definidas a cima
df_repetidos_apagados_hospitais = df_repetidos_apagados_hospitais[colunas_apagados]

del colunas_apagados, df_repetidos_apagados, 


In [83]:
# Cria uma lista chamada hospitais com todos os nomes dos hospitais que aparecem na planilha df_repetidos_apagados_hospitais
hospitais = df_repetidos_apagados_hospitais.ID_UNIDADE.unique()

# Cria uma variável rota com a rota onde a planilha será salva, caso deseje musar isto, alterar aqui
rota = 'bases/duplicidades/duplicidades por hospitais/'

# Cria uma variável extencao que serve para definir a extensão da planilha a ser salva
extencao = '.xlsx'

# Cria um loop que irá ocorrer a quantidade de hospitais na planilha df_repetidos_apagados_hospitais
for x in range(hospitais.shape[0]):
    # Cria uma variável nome_hospital com o nome do hospital que está sendo percorrido pela lista
    nome_hospital = hospitais[x]

    # Cria uma variável salvar que junta a rota, nome do hospital e a extensão Ex: bases/planilha dos hospitais apagar/HRAN.xlsx
    salvar = rota+nome_hospital+extencao

    # Na posição da lista que o loop está percorrendo, é feito um df somente com os registros de repetidos a serem apagados do hospital e
    # salva uma planilha com o nome do hospital na rota criada em rota
    df_repetidos_apagados_hospitais.loc[df_repetidos_apagados_hospitais['ID_UNIDADE']==hospitais[x]].to_excel(salvar)

In [84]:
# Após gerar as planilhas, reatribui a df a planilha original com as datas no formato datetime
df = dff

# Deixa no df somente os registros sem duplicidades, salvo os casos de reinternação
df = df.loc[df['REINTERNACAO']!=2]

# Apaga variáveis não mais utilizadas
del df_repetidos_apagados_hospitais, dff, extencao, hospitais, nome_hospital, 

# 3 - Detectar Registros Inconsistentes ou Incompletos

In [85]:
# Cria uma lista com nome de novas colunas para definir os erros
colunas_erros = ['HOMEM_PUERPERO', 'HOMEM_GRAVIDO', 'PREENCHER_UTI', 'PREENCHER_DATA_ENTRADA_UTI', 'PREENCHER_DATA_SAIDA_UTI', 
'OBITOS_SEM_HOSPITALIZACAO','CRIETERIO_4_TOMO_RES_DIFERENTE_1_OU_5', 'PREENCHER_HOSPITALIZACAO', 'CURA_SEM_HOSPITALIZACAO', 
'PREENCHER_SUPORT_VEN', 'PREENCHER_CLASSI_OUT_VAZIA_SE_CLASSI_FIN_3', 'CLASSI_FIN_5_EVOLUCAO_2_E_CRITERIO_3', 'ENCERRADOS_IMCOMPLETOS', 
'PREENCHER_DT_ENCERRA', 'PREENCHER_CLASSI_FIN', 'PREENCHER_EVOLUCAO', 'CLASSI_FIN_4_CRITERIO_2_OU_4', 'SEM_SINTOMA', 'SRAG_LABORATORIAL_ERRADO', 'CLASSI_FIN_1_COVID_1']

# Cria as colunas de erros em df e seta o valor 0 para todas as linhas das novas colunas
df[colunas_erros] = 0

# Redefine o index de df
df = df.reset_index()

## 3.1.1 - Identifica Homens classificados como Puerpera
Nos registros onde o paciente é do sexo masculino e classificação "Sim" para puerpera, será escrito "Sim" na coluna 'HOMEM_PUERPERO'

In [86]:
# Homens classificados como Puérpera
'HOMEM_PUERPERO'

# Cria uma lista com os index onde existe homem classificado como puérpera 
index_list = df.loc[df['CS_SEXO'] =='M'].loc[df['PUERPERA']==1,'HOMEM_PUERPERO'].index

# Escreve 'Sim' na coluna HOMEM_PUERPERO onde existe homeme classificado como puérpero
df.loc[df.index[index_list],'HOMEM_PUERPERO'] = 'Sim'

## 3.1.2 - Identifica Homens classificados como Gravida
Nos registros onde o paciente é do sexo masculino e classificação "Sim" para Gravida, será escrito "Sim" na coluna 'HOMEM_GRAVIDO'

In [87]:
# Homens que não são classificados como CS_GESTANT (6), ignorado (9) ou 5 = Não
'HOMEM_GRAVIDO'

# Cria uma planilha somente com os registros de homens
df_homem_gravido = df.loc[df['CS_SEXO']=='M']

#Cria uma planilha derivada da planilha de homens, onde o código CS_GESTANT é 1
df_homem_gravido1 = df_homem_gravido.loc[df['CS_GESTANT']==1]

#Cria uma planilha derivada da planilha de homens, onde o código CS_GESTANT é 2
df_homem_gravido2 = df_homem_gravido.loc[df['CS_GESTANT']==2]

#Cria uma planilha derivada da planilha de homens, onde o código CS_GESTANT é 3
df_homem_gravido3 = df_homem_gravido.loc[df['CS_GESTANT']==3]

#Cria uma planilha derivada da planilha de homens, onde o código CS_GESTANT é 4
df_homem_gravido4 = df_homem_gravido.loc[df['CS_GESTANT']==4]

# Junta as colunas criadas a cima com cada código
df_homem_gravido = pd.concat([df_homem_gravido1, df_homem_gravido2, df_homem_gravido3, df_homem_gravido4])

# Cria uma lista com os index nos registros de homens classificados como grávidos
index_list = df_homem_gravido.index

# Escreve 'Sim' na coluna HOMEM_GRAVIDO onde existe homeme classificado como grávidos
df.loc[df.index[index_list],'HOMEM_GRAVIDO'] = 'Sim'

del df_homem_gravido


## 3.1.3 - Identifica casos onde a UTI está Ignorada ou nula
Nos registros onde estiver 9 (ignorado) ou nulo na coluna UTI, será escrito "Sim" na coluna 'PREENCHER_UTI'

In [88]:
# UTI = 9 ou nula
'PREENCHER_UTI'

# Cria a planilha df_inconsistentes_UTI onde UTI está nula ou 9
df_inconsistentes_UTI = pd.concat([df[df['UTI'].isna()],df.loc[df['UTI']==9]])

# Cria a coluna SUBTRACAO_MES_ATUAL na planilha df_inconsistentes_UTI
df_inconsistentes_UTI['SUBTRACAO_MES_ATUAL'] = 0

# Cria um loop que vai percorrer os valores de DT_SIN_PRI em df_inconsistentes_UTI e subtrair pelo mês atual, oresultado da subtração aloca na variável SUBTRACAO_MES_ATUAL
for x in df_inconsistentes_UTI.index:
    df_inconsistentes_UTI['SUBTRACAO_MES_ATUAL'][x] = (datetime.now() - df_inconsistentes_UTI['DT_SIN_PRI'][x]).days/30

# Deixa na planilha df_inconsistentes_UTI somente valores que possuem 6 ou + meses de distância da data atual
df_inconsistentes_UTI = df_inconsistentes_UTI.loc[df_inconsistentes_UTI['SUBTRACAO_MES_ATUAL'] >= 6]

# Cria uma lista com os index nos registros com mais de 6 meses e UTI está nula ou 9
index_list = df_inconsistentes_UTI.index

# Escreve 'Sim' na coluna PREENCHER_UTI nos registros com mais de 6 meses e UTI está nula ou 9
df.loc[df.index[index_list],'PREENCHER_UTI'] = 'Sim'

## 3.1.4 - Identifica Casos de UTI sem Data de Entrada
Nos registros onde UTI = 1 (Sim) e a data de entrada na UTI esta nula, será escrito "Sim" na coluna 'PREENCHER_DATA_ENTRADA_UTI'

In [89]:
# UTI = 1 e sem data de entrada UTI
'PREENCHER_DATA_ENTRADA_UTI'

# Cria uma lista com os index onde existe homem classificado como puérpera 
index_list = df.loc[df['UTI'] ==1].loc[df['DT_ENTUTI'].isna(),'PREENCHER_DATA_ENTRADA_UTI'].index

# Escreve 'Sim' na coluna PREENCHER_DATA_ENTRADA_UTI onde não existe data de entrada UTI
df.loc[df.index[index_list],'PREENCHER_DATA_ENTRADA_UTI'] = 'Sim'

## 3.1.5 - Identifica Casos de UTI sem Data de Saída, com mais de 6 meses de primeiros sintomas
Nos registros com mais de 6 meses da data de primeiros sintomas onde UTI = 1 (Sim) e a data de saída na UTI esta nula, será escrito "Sim" na coluna 'PREENCHER_DATA_SAIDA_UTI'

In [90]:
# UTI = 1 e sem data de saída UTI
'PREENCHER_DATA_SAIDA_UTI'

# Cria uma lista com os index onde existe homem classificado como puérpera 
df_UTI_sem_data_saida = df.loc[df['UTI'] ==1].loc[df['DT_SAIDUTI'].isna()]



# Cria a coluna SUBTRACAO_MES_ATUAL na planilha df_UTI_sem_data_saida 
df_UTI_sem_data_saida['SUBTRACAO_MES_ATUAL'] = 0

# Cria um loop que vai percorrer os valores de DT_SIN_PRI em df_UTI_sem_data_saida e subtrair pelo mês atual, o resultado da subtração aloca na variável SUBTRACAO_MES_ATUAL
for x in df_UTI_sem_data_saida.index:
    df_UTI_sem_data_saida['SUBTRACAO_MES_ATUAL'][x] = (datetime.now() - df_UTI_sem_data_saida['DT_SIN_PRI'][x]).days/30

# Deixa na planilha df_inconsistentes_UTI somente valores que possuem 6 ou + meses de distância da data atual
df_UTI_sem_data_saida = df_UTI_sem_data_saida.loc[df_UTI_sem_data_saida['SUBTRACAO_MES_ATUAL'] >= 6]

# Cria uma lista com os index nos registros com mais de 6 meses e UTI está nula ou 9
index_list = df_UTI_sem_data_saida.index

# Escreve 'Sim' na coluna PREENCHER_UTI nos registros com mais de 6 meses e UTI está nula ou 9
df.loc[df.index[index_list],'PREENCHER_DATA_SAIDA_UTI'] = 'Sim'

## 3.1.6 - Identificar casos de Óbitos sem hospitalização
Nos registros onde o pacienteveio a óbito e classificação não para Hospitalização, será escrito "Sim" na coluna 'OBITOS_SEM_HOSPITALIZACAO'

In [91]:
# Óbitos sem hospitalização
'OBITOS_SEM_HOSPITALIZACAO'

# Cria uma planilha somente com os registros de não hospitalizados
df_nao_hospitalizacao_obito = df.loc[df['HOSPITAL']==2]

#Cria uma planilha de não hospitalizados e vieram a óbito por covid
df_nao_hospitalizacao_obito1 = df_nao_hospitalizacao_obito.loc[df['EVOLUCAO']==2]

#Cria uma planilha de não hospitalizados e vieram a óbito por outras causas
df_nao_hospitalizacao_obito2 = df_nao_hospitalizacao_obito.loc[df['EVOLUCAO']==3]

# Junta as colunas criadas a cima
df_nao_hospitalizacao_obito = pd.concat([df_nao_hospitalizacao_obito1, df_nao_hospitalizacao_obito2,])

# Cria uma lista com os index nos registros com mais de 6 meses e UTI está nula ou 9
index_list = df_nao_hospitalizacao_obito.index

# Escreve 'Sim' na coluna OBITOS_SEM_HOSPITALIZACAO nos registros com mais de 6 meses e UTI está nula ou 9
df.loc[df.index[index_list],'OBITOS_SEM_HOSPITALIZACAO'] = 'Sim'

## 3.1.7 - Identifica Casos onde o Critério de finalização é 4 e o resultado da tomografia é diferente de 1 ou 5
Nos registros o critério de finalização é 4 e o resultado da tomografia é diferente de 1 ou 5, será escrito "Sim" na coluna 'CRIETERIO_4_TOMO_RES_DIFERENTE_1_OU_5'

In [92]:
# CRITERIO = 4 e TOMO_RES diferente de 1 ou 5
'CRIETERIO_4_TOMO_RES_DIFERENTE_1_OU_5'

# Cria uma lista com os index onde critério = 4, TOMO_RES diferente de 1 e 5
index_list = df.loc[df['CRITERIO'] ==4].loc[df['TOMO_RES']!=1].loc[df['TOMO_RES']!=5].index

# Escreve 'Sim' na coluna CRIETERIO_4_TOMO_RES_DIFERENTE_1_OU_5 onde CRITERIO = 4 e TOMO_RES diferente de 1 ou 5
df.loc[df.index[index_list],'CRIETERIO_4_TOMO_RES_DIFERENTE_1_OU_5'] = 'Sim'


## 3.1.8 - Identifica os casos onde possui hospitalização = 9 (ignorada) ou nula
Nos registros onde onde possui hospitalização = 9 (ignorada) ou nula, será escrito "Sim" na coluna 'PREENCHER_HOSPITALIZACAO'

In [93]:
# HOSPITAL = ignorado ou vazio
'PREENCHER_HOSPITALIZACAO'

# CRIA UMA PLANILHA COM OS REGISTROS ONDE HOSPITAL = 9 OU NULO
df_sem_hospitalizacao = pd.concat([df[df['HOSPITAL'].isna()],df.loc[df['HOSPITAL'] == 9]])

# CRIA UMA lista COM OS index onde os REGISTROS HOSPITAL = 9 OU NULO
index_list = df_sem_hospitalizacao.index

# Escreve 'Sim' na coluna PREENCHER_HOSPITALIZACAO onde HOSPITAL = ignorado ou vazio
df.loc[df.index[index_list],'PREENCHER_HOSPITALIZACAO'] = 'Sim'

## 3.1.9 - Identifica casos onde a pessoa foi curada e não possui hospitalização
Nos registros onde a pessoa foi curada e não possui hospitalização, será escrito "Sim" na coluna 'CURA_SEM_HOSPITALIZACAO'

In [94]:
# HOSPITAL = 2 e EVOLUCAO = 1
'CURA_SEM_HOSPITALIZACAO'

# Cria uma lista com os index onde critério = 4, TOMO_RES diferente de 1 e 5
index_list = df.loc[df['HOSPITAL'] == 2].loc[df['EVOLUCAO']==1].index

# Escreve 'Sim' na coluna CURA_SEM_HOSPITALIZACAO onde HOSPITAL = 2 e EVOLUCAO = 1
df.loc[df.index[index_list],'CURA_SEM_HOSPITALIZACAO'] = 'Sim'

## 3.1.10 - Identifica Registros com valor 9 (ignorado) ou nulo em Suporte ventilatório
Nos registros com valor 9 (ignorado) ou nulo em Suporte ventilatório, será escrito "Sim" na coluna 'PREENCHER_SUPORT_VEN'

In [95]:
# SUPORTE_VEN = 9 ou NAN
'PREENCHER_SUPORT_VEN'

# Cria uma planilha onde SUPORTE_VEN = 9 ou NAN
df_sem_suporte_vent = pd.concat([df[df['SUPORT_VEN'].isna()],df.loc[df['SUPORT_VEN'] == 9]])

# Cria uma lista com os index onde SUPORTE_VEN = 9 ou NAN
index_list = df_sem_suporte_vent.index

# Escreve 'Sim' na coluna CURA_SEM_HOSPITALIZACAO onde SUPORTE_VEN = 9 ou NAN
df.loc[df.index[index_list],'PREENCHER_HOSPITALIZACAO'] = 'Sim'

## 3.1.11 - Identifica registros com Classificação final = 3 e CLASSI_OUT nula
Nos registros com Classificação final = 3 e CLASSI_OUT nula, será escrito "Sim" na coluna 'PREENCHER_CLASSI_OUT_VAZIA_SE_CLASSI_FIN_3'

In [96]:
# CLASSI_FIN = 3 E CLASSI_OUT = NAN
'PREENCHER_CLASSI_OUT_VAZIA_SE_CLASSI_FIN_3'

# Cria uma lista com os index onde CLASSI_FIN = 3 E CLASSI_OUT = NAN
index_list = df.loc[df['CLASSI_FIN'] == 3].loc[df['CLASSI_OUT'].isna()].index

# Escreve 'Sim' na coluna PREENCHER_CLASSI_OUT_VAZIA_SE_CLASSI_FIN_3 onde CLASSI_FIN = 3 E CLASSI_OUT = NAN
df.loc[df.index[index_list],'PREENCHER_CLASSI_OUT_VAZIA_SE_CLASSI_FIN_3'] = 'Sim'

## 3.1.12 - Identifica registros com Classificação final = 5, Evolução = 2 e Critério = 3
Nos registros onde Classificação final = 5, Evolução = 2 e Critério = 3, será escrito "Sim" na coluna 'CLASSI_FIN_5_EVOLUCAO_2_E_CRITERIO_3'

In [97]:
# CLASSI_FIN = 5 EVOLUCAO = 2 E CRITERIO = 3
'CLASSI_FIN_5_EVOLUCAO_2_E_CRITERIO_3'

# Cria uma lista com os index onde CLASSI_FIN = 5 EVOLUCAO = 2 E CRITERIO = 3
index_list = df.loc[df['CLASSI_FIN'] == 5].loc[df['EVOLUCAO'] == 2].loc[df['CRITERIO'] == 3].index

# Escreve 'Sim' na coluna PREENCHER_CLASSI_OUT_VAZIA_SE_CLASSI_FIN_3 onde CLASSI_FIN = 5 E EVOLUCAO = 2 E CRITERIO = 3
df.loc[df.index[index_list],'CLASSI_FIN_5_EVOLUCAO_2_E_CRITERIO_3'] = 'Sim'

## 3.1.13 - Identifica registros com data de encerrados e sem Classificação final, Critério de encerramento, Evolução ou data de Evolução
Nos registros com data de encerrados e sem Classificação final, Critério de encerramento, Evolução ou data de Evolução, será escrito o que falta prencher na coluna 'ENCERRADOS_IMCOMPLETOS'

In [98]:
'ENCERRADOS_IMCOMPLETOS'

# CRIA UMA PLANILHA df_encerrados_incompletos COM REGISTROS NÃO NULOS EM DT_ENCERRA
df_encerrados_incompletos = df[df['DT_ENCERRA'].notnull()]

# ADICIONA 0 NA COLUNA ENCERRADOS_IMCOMPLETOS
df_encerrados_incompletos['ENCERRADOS_IMCOMPLETOS'] = 0 

# PREENCHE NA COLUNA  ENCERRADOS_IMCOMPLETOS Preencher CLASSI_FIN SE CLASSI_FIN ESTIVER NULO
df_encerrados_incompletos.loc[df_encerrados_incompletos['CLASSI_FIN'].isnull(),'ENCERRADOS_IMCOMPLETOS'] = 'Preencher CLASSI_FIN'

# PREENCHE NA COLUNA  ENCERRADOS_IMCOMPLETOS 'Preencher CRITERIO' SE CRITERIO ESTIVER NULO
df_encerrados_incompletos.loc[df_encerrados_incompletos['CRITERIO'].isnull(),'ENCERRADOS_IMCOMPLETOS'] = 'Preencher CRITERIO'

# PREENCHE NA COLUNA  ENCERRADOS_IMCOMPLETOS 'Preencher EVOLUCAO' SE EVOLUCAO ESTIVER NULO
df_encerrados_incompletos.loc[df_encerrados_incompletos['EVOLUCAO'].isnull(),'ENCERRADOS_IMCOMPLETOS'] = 'Preencher EVOLUCAO'

# PREENCHE NA COLUNA  ENCERRADOS_IMCOMPLETOS 'Preencher DT_EVOLUCA' SE DT_EVOLUCA ESTIVER NULO
df_encerrados_incompletos.loc[df_encerrados_incompletos['DT_EVOLUCA'].isnull(),'ENCERRADOS_IMCOMPLETOS'] = 'Preencher DT_EVOLUCA'   

# Deixa somente os registros onde DT_ENCERRA não é nulo e tem ao menos um dos campos (75,76,77 ou 78) nulos
df_encerrados_incompletos = df_encerrados_incompletos.loc[df_encerrados_incompletos['ENCERRADOS_IMCOMPLETOS']!=0]

# Cria uma lista com os index de df_encerrados_incompletos
index_list = df_encerrados_incompletos.index

# Escreve o critério na coluna ENCERRADOS_IMCOMPLETOS nos REGISTROS NÃO NULOS EM DT_ENCERRA e sem critérios
df.loc[df.index[index_list],'ENCERRADOS_IMCOMPLETOS'] = df_encerrados_incompletos['ENCERRADOS_IMCOMPLETOS']

## 3.1.14 - Identifica registros com mais de 6 meses da data de início dos primeiros sintomas e sem data de encerramento
Nos registros com mais de 6 meses da data de início dos primeiros sintomas e sem data de encerramento, será escrito "Sim" na coluna 'PREENCHER_DT_ENCERRA'

In [99]:
# 79 = nan
'PREENCHER_DT_ENCERRA'

# CRIA UMA PLANILHA df_sem_dt_encerramento COM VALORES ONDE DT_ENCERRA É NULA
df_sem_dt_encerramento = df[df['DT_ENCERRA'].isna()]

# Cria a coluna SUBTRACAO_MES_ATUAL na planilha df_sem_dt_encerramento 
df_sem_dt_encerramento['SUBTRACAO_MES_ATUAL'] = 0

# Cria um loop que vai percorrer os valores de DT_SIN_PRI em df_sem_dt_encerramento e subtrair pelo mês atual, o resultado da subtração aloca na variável SUBTRACAO_MES_ATUAL
for x in df_sem_dt_encerramento.index:
    df_sem_dt_encerramento['SUBTRACAO_MES_ATUAL'][x] = (datetime.now() - df_sem_dt_encerramento['DT_SIN_PRI'][x]).days/30

# Deixa na planilha df_inconsistentes_UTI somente valores que possuem 6 ou + meses de distância da data atual
df_sem_dt_encerramento = df_sem_dt_encerramento.loc[df_sem_dt_encerramento['SUBTRACAO_MES_ATUAL'] >= 6]

# Cria uma lista com os index nos registros com mais de 6 meses e 79 = nan
index_list = df_sem_dt_encerramento.index

# Escreve 'Sim' na coluna PREENCHER_UTI nos registros com mais de 6 meses e 79 = nan
df.loc[df.index[index_list],'PREENCHER_DT_ENCERRA'] = 'Sim'

## 3.1.15 - Identifica registros com mais de 6 meses da data de início dos primeiros sintomas e sem classificação final
Nos registros com mais de 6 meses da data de início dos primeiros sintomas e sem classificação final, será escrito "Sim" na coluna 'PREENCHER_CLASSI_FIN'

In [100]:
# 75= nan
'PREENCHER_CLASSI_FIN'

# CRIA UMA PLANILHA df_sem_classi_fim COM VALORES ONDE DT_ENCERRA É NULA
df_sem_classi_fim = df[df['CLASSI_FIN'].isna()]

# Cria a coluna SUBTRACAO_MES_ATUAL na planilha df_sem_classi_fim 
df_sem_classi_fim['SUBTRACAO_MES_ATUAL'] = 0

# Cria um loop que vai percorrer os valores de DT_SIN_PRI em df_sem_classi_fim e subtrair pelo mês atual, o resultado da subtração aloca na variável SUBTRACAO_MES_ATUAL
for x in df_sem_classi_fim.index:
    df_sem_classi_fim['SUBTRACAO_MES_ATUAL'][x] = (datetime.now() - df_sem_classi_fim['DT_SIN_PRI'][x]).days/30

# Deixa na planilha df_inconsistentes_UTI somente valores que possuem 6 ou + meses de distância da data atual
df_sem_classi_fim = df_sem_classi_fim.loc[df_sem_classi_fim['SUBTRACAO_MES_ATUAL'] >= 6]

# Cria uma lista com os index nos registros com mais de 6 meses 75= nan
index_list = df_sem_classi_fim.index

# Escreve 'Sim' na coluna PREENCHER_UTI nos registros com mais de 6 meses e 75= nan
df.loc[df.index[index_list],'PREENCHER_CLASSI_FIN'] = 'Sim'

## 3.1.16 - Identifica registros com mais de 6 meses da data de início dos primeiros sintomas e sem EVOLUÇÂO (nula)
Nos registros com mais de 6 meses da data de início dos primeiros sintomas e sem EVOLUÇÂO (nula), será escrito "Sim" na coluna 'PREENCHER_EVOLUCAO'

In [101]:
# 77 = NAN
'PREENCHER_EVOLUCAO'

# CRIA UMA PLANILHA df_sem_evolucao COM VALORES ONDE DT_ENCERRA É NULA
df_sem_evolucao = df[df['EVOLUCAO'].isna()]

# Cria a coluna SUBTRACAO_MES_ATUAL na planilha df_sem_evolucao 
df_sem_evolucao['SUBTRACAO_MES_ATUAL'] = 0

# Cria um loop que vai percorrer os valores de DT_SIN_PRI em df_sem_evolucao e subtrair pelo mês atual, o resultado da subtração aloca na variável SUBTRACAO_MES_ATUAL
for x in df_sem_evolucao.index:
    df_sem_evolucao['SUBTRACAO_MES_ATUAL'][x] = (datetime.now() - df_sem_evolucao['DT_SIN_PRI'][x]).days/30

# Deixa na planilha df_inconsistentes_UTI somente valores que possuem 6 ou + meses de distância da data atual
df_sem_evolucao = df_sem_evolucao.loc[df_sem_evolucao['SUBTRACAO_MES_ATUAL'] >= 6]

# Cria uma lista com os index nos registros com mais de 6 meses e 77 = NAN
index_list = df_sem_evolucao.index

# Escreve 'Sim' na coluna PREENCHER_UTI nos registros com mais de 6 meses e 77 = NAN
df.loc[df.index[index_list],'PREENCHER_EVOLUCAO'] = 'Sim'

## 3.1.17 - Identifica registros com Classificação Final = 4 e critério = 2 ou 4
Nos registros com Classificação Final = 4 e critério = 2 ou 4, será escrito "Sim" na coluna 'CLASSI_FIN_4_CRITERIO_2_OU_4'

In [102]:
# 75 = 4 E 76 = 2 OU 4
'CLASSI_FIN_4_CRITERIO_2_OU_4'

# Cria uma planilha classi_fin4_criteirio_2_ou_4 com os casos onde classi_fin é 4
classi_fin4_criteirio_2_ou_4 = df.loc[df['CLASSI_FIN']==4]

# CRIA UMA PLANILHA classi_fin4_criteirio_2 ONDE CLASSI_FIN É 4 E CRITERIO = 2
classi_fin4_criteirio_2 = classi_fin4_criteirio_2_ou_4.loc[classi_fin4_criteirio_2_ou_4['CRITERIO']==2]

# CRIA UMA PLANILHA classi_fin4_criteirio_4 ONDE CLASSI_FIN É 4 E CRITERIO = 4
classi_fin4_criteirio_4 = classi_fin4_criteirio_2_ou_4.loc[classi_fin4_criteirio_2_ou_4['CRITERIO']==4]

# jUNTA AS PLANILHAS classi_fin4_criteirio_2 E classi_fin4_criteirio_4 NA PLANILHA classi_fin4_criteirio_2_ou_4
classi_fin4_criteirio_2_ou_4 = pd.concat([classi_fin4_criteirio_2,classi_fin4_criteirio_4])

# Cria uma lista com os index nos registros  75 = 4 E 76 = 2 OU 4
index_list = classi_fin4_criteirio_2_ou_4.index

# Escreve 'Sim' na coluna PREENCHER_UTI nos registros  75 = 4 E 76 = 2 OU 4
df.loc[df.index[index_list],'CLASSI_FIN_4_CRITERIO_2_OU_4'] = 'Sim'

## 3.1.18 - Identifica os registros que não possuem sintomas
Nos registros que não possuem sintomas, será escrito "Sim" na coluna 'SEM_SINTOMA'

In [103]:
# Registros sem nenhum sintma
'SEM_SINTOMA'

# CRIA UMA LISTA COM as colunas dos sintomas
colunas_sintomas = ['FEBRE','TOSSE','GARGANTA','DISPNEIA','DESC_RESP','SATURACAO','DIARREIA','VOMITO','DOR_ABD','FADIGA','PERD_OLFT','PERD_PALA','OUTRO_SIN','OUTRO_DES']

# Cria uma planilha df_sem_sintomas com os valores de df
df_sem_sintomas = df

# Aloca na coluna SEM_SINTOMA a quantidade de sintomas nulos de cada linha
df_sem_sintomas['SOMA'] = df_sem_sintomas[colunas_sintomas].isnull().sum(axis=1)

# Deixa na planilha df_sem_sintomas somente os registro onde a coluna SEM_SINTOMA = 14 (quantidade de colunas de sintomas)
df_sem_sintomas = df_sem_sintomas.loc[df_sem_sintomas['SOMA'] == 14]

# Cria uma lista com os index dos registros sem sintomas
index_list = df_sem_sintomas.index

# Escreve 'Sim' na coluna PREENCHER_UTI nos registros sem nenhum sintma
df.loc[df.index[index_list],'SEM_SINTOMA'] = 'Sim'

## 3.1.19 - Identifica os registros classificados como SRAG Laboratorial mas não possuem informações dos exames
Nos registros classificados como SRAG Laboratorial mas não possuem informações dos exames, será escrito "Preencher colunas PCR_SARS2, AN_SARS2, RES_IGG ou RES_IGM" na coluna 'SRAG_LABORATORIAL_ERRADO'

In [104]:
# 75 = 5 E 76 = 1 E PCR_SARS2 e AN_SARS2 = NAN e RES_IGG e RES_IGM = NAN e !=1

'SRAG_LABORATORIAL_ERRADO'

# Cria uma lista com os index onde existe 75 = 5 E 76 = 1 E PCR_SARS2 e AN_SARS2 = NAN e RES_IGG e RES_IGM = NAN e !=1
index_list = df.loc[df['CLASSI_FIN'] ==5].loc[df['CRITERIO']==1].loc[df['PCR_SARS2'].isna()].loc[df['AN_SARS2'].isna()].loc[df['RES_IGG'].isna()].loc[df['RES_IGM'].isna()].index

# Escreve 'Preencher colunas PCR_SARS2, AN_SARS2, RES_IGG ou RES_IGM' na coluna HOMEM_PUERPERO onde existe homeme classificado como puérpero
df.loc[df.index[index_list],'SRAG_LABORATORIAL_ERRADO'] = 'Preencher colunas PCR_SARS2, AN_SARS2, RES_IGG ou RES_IGM'

## 3.1.20 - Identifica os registros classificados como influenza e possuem exames resultados para Covid-19
Nos registros classificados como influenza e possuem exames resultados para Covid-19, será escrito "Sim" na coluna 'CLASSI_FIN_1_COVID_1'

In [105]:
# Campo 75 CLASSI_FIN influenza = 1 --> PCR_SARS2 ou AN_SARS2 = 1 ou RES_IGG ou RES_IGM = 1

'CLASSI_FIN_1_COVID_1'

# Cria uma planilha df_inflenza_e_covid com os registros onde CLASSI_FIN = 1
df_inflenza_e_covid = df.loc[df['CLASSI_FIN']==1]

# Cria uma planilha df_inflenza_e_covid_PCR_SARS2 com os registros de df_inflenza_e_covid onde PCR_SARS2 = 1
df_inflenza_e_covid_PCR_SARS2 = df_inflenza_e_covid.loc[df_inflenza_e_covid['PCR_SARS2']==1]

# Cria uma planilha df_inflenza_e_covid_AN_SARS2 com os registros de df_inflenza_e_covid onde AN_SARS2 = 1
df_inflenza_e_covid_AN_SARS2 = df_inflenza_e_covid.loc[df_inflenza_e_covid['AN_SARS2']==1]

# Cria uma planilha df_inflenza_e_covid_RES_IGG com os registros de df_inflenza_e_covid onde RES_IGG = 1
df_inflenza_e_covid_RES_IGG = df_inflenza_e_covid.loc[df_inflenza_e_covid['RES_IGG']==1]

# Cria uma planilha df_inflenza_e_covid_RES_IGM com os registros de df_inflenza_e_covid onde RES_IGM = 1
df_inflenza_e_covid_RES_IGM = df_inflenza_e_covid.loc[df_inflenza_e_covid['RES_IGM']==1]

# Junta as 4 planilhas criadas a cima e aloca na planilha df_inflenza_e_covid
df_inflenza_e_covid = pd.concat([df_inflenza_e_covid_PCR_SARS2, df_inflenza_e_covid_AN_SARS2, df_inflenza_e_covid_RES_IGG, df_inflenza_e_covid_RES_IGM])

# Cria uma lista com os index dos registros 75 CLASSI_FIN influenza = 1 --> PCR_SARS2 ou AN_SARS2 = 1 ou RES_IGG ou RES_IGM = 1
index_list = df_inflenza_e_covid.index

# Escreve 'Sim' na coluna PREENCHER_UTI nos registros 75 CLASSI_FIN influenza = 1 --> PCR_SARS2 ou AN_SARS2 = 1 ou RES_IGG ou RES_IGM = 1
df.loc[df.index[index_list],'CLASSI_FIN_1_COVID_1'] = 'Sim'

# 3.2.1 Salvar Planilhas de inconsistências

In [106]:
# Define quais serão as colunas de erros
colunas_finais = ['ID_UNIDADE','HOMEM_PUERPERO', 'HOMEM_GRAVIDO', 'PREENCHER_UTI', 'PREENCHER_DATA_ENTRADA_UTI', 'PREENCHER_DATA_SAIDA_UTI', 
'OBITOS_SEM_HOSPITALIZACAO','CRIETERIO_4_TOMO_RES_DIFERENTE_1_OU_5', 'PREENCHER_HOSPITALIZACAO', 'CURA_SEM_HOSPITALIZACAO', 
'PREENCHER_SUPORT_VEN', 'PREENCHER_CLASSI_OUT_VAZIA_SE_CLASSI_FIN_3', 'CLASSI_FIN_5_EVOLUCAO_2_E_CRITERIO_3', 'ENCERRADOS_IMCOMPLETOS', 
'PREENCHER_DT_ENCERRA', 'PREENCHER_CLASSI_FIN', 'PREENCHER_EVOLUCAO', 'CLASSI_FIN_4_CRITERIO_2_OU_4', 'SEM_SINTOMA', 'SRAG_LABORATORIAL_ERRADO', 'CLASSI_FIN_1_COVID_1']

In [107]:
# transforma a coluna NU_NOTIFIC em index
df.set_index('NU_NOTIFIC', inplace = True)

# Substitui os valores zeros por nulos
df.replace(0, np.nan, inplace=True)

# Aloca na coluna ERROS a quantidade de ERROS de cada linha
df['ERROS'] = df[colunas_erros].isnull().sum(axis=1)

# Deixa na planilha df somente os registros que possuem ao menos 1 erro
df = df.loc[df['ERROS'] != 20]

# Deixa somente as colunas de erros e de ID_UNIDADE
df = df[colunas_finais]

# Substitui os valores nulos por '--'
df.replace(np.nan, '--', inplace=True)

In [108]:
# Salva a planilha com o nome dos hospitais e os erros
df.to_excel("bases/erros/todos_erros.xlsx")

# 3.2.2 Salvar Planilhas de inconsistências por Hospital


In [110]:
# Cria uma lista chamada hospitais com todos os nomes dos hospitais que aparecem na planilha df
hospitais = df.ID_UNIDADE.unique()

# Cria uma variável rota com a rota onde a planilha será salva, caso deseje musar isto, alterar aqui
rota = 'bases/erros/erros por hospitais/'

# Cria uma variável extencao que serve para definir a extensão da planilha a ser salva
extencao = '.xlsx'

# Cria um loop que irá ocorrer a quantidade de hospitais na planilha df
for x in range(hospitais.shape[0]):
    # Cria uma variável nome_hospital com o nome do hospital que está sendo percorrido pela lista
    nome_hospital = hospitais[x]

    # Cria uma variável salvar que junta a rota, nome do hospital e a extensão Ex: bases/planilha dos hospitais apagar/HRAN.xlsx
    salvar = rota+nome_hospital+extencao

    # Na posição da lista que o loop está percorrendo, é feito um df somente com os registros de repetidos a serem apagados do hospital e
    # salva uma planilha com o nome do hospital na rota criada em rota
    df_hospital = df.loc[df['ID_UNIDADE']==hospitais[x]]

    # troca os 2 tracinhos por um valor nulo
    df_hospital.replace('--', np.nan, inplace=True)

    # Apaga as colunas onde todos os valores são nulos
    df_hospital.dropna(axis=1, how='all', inplace = True)

    # Salva a planilha em excel 
    df_hospital.to_excel(salvar)

# 4 Envio de e-mails 

In [111]:
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email import encoders
import ast

# 4.1 - Enviar e-mails com casos Duplicados

In [112]:
# Lê o arquivo com as duplicidades a serem apagadas e aloca na variável hospitais
hospitais = pd.read_excel("bases/duplicidades/duplicidades_apagadas.xlsx")

# Transforma a variável hospitais em um array que contém os nomes dos hospitais presentes na planilha de duplicidades a serem apagadas
hospitais = hospitais.ID_UNIDADE.unique()

In [113]:
# É necessário que exista, na mesma pasta do código) um arquivo 'lista_emails.txt' contento um dicionário com o nome dos hospitais (como digitado nas planilhas) e o e-mail do hospital
# Instancia o arquivo com os emails dos Hospitais
file  = open("lista_emails.txt", 'r')

# Lê o arquivo de emails e adiciona na variável contents
contents = file.read()

# Cria um dicionário chamado emails, com o nome dos hospitais e os emails - o que está na lista_emails.txt
emails = ast.literal_eval(contents)

In [ ]:
# Cria uma variável rota com a rota onde a planilha será salva, caso deseje musar isto, alterar aqui
rota = 'bases/duplicidades/duplicidades por hospitais/'

# Cria uma variável extencao que serve para definir a extensão da planilha a ser salva
extencao = '.xlsx'

username = 'apcs.1502@gmail.com'
password = 'fomgnsjujrxcrfpc'
mail_from  = 'apcs.1502@gmail.com'

título_email = 'Registros duplicados - '
corpo_email = 'Segue em anexo os registros que identificamos com duplicados, por gentileza apague todos do sistema.'


# Cria uma lista com todos os e-mails presentes de hospitais na lista de e-mails - evita que de erro ao tentar enviar um e-mail para um hospital que não esteja na lista
lista_todos_emails = list()
for i in emails.keys():
    lista_todos_emails.append(i)


# Cria um loop que irá ocorrer a quantidade de hospitais na planilha df_repetidos_apagados_hospitais
for x in range(hospitais.shape[0]):
    # Cria uma variável nome_hospital com o nome do hospital que está sendo percorrido pela lista
    nome_hospital = hospitais[x]

    # Verifica se o nome do hospital tem e-mail
    if nome_hospital in lista_todos_emails:

        # Cria uma variável salvar que junta a rota, nome do hospital e a extensão Ex: bases/planilha dos hospitais apagar/HRAN.xlsx
        salvar = rota+nome_hospital+extencao

        # Seleciona pra qual e-mail a mensagem e a planilha serão enviados
        mail_to = emails[nome_hospital]
        # Título do e-mail
        mail_subject = título_email+nome_hospital
        # Corpo do e-mail
        mail_body = 'Favor apagar no sistema SIVEP o registro dos pacientes na planilha em anexo'

        # Arquivo a ser enviado (passa a rota do arquivo de cada hospital)
        mail_attachment=salvar
        # Nome e extensão do arquivo
        mail_attachment_name=nome_hospital+extencao

        mimemsg = MIMEMultipart()
        # seleciona qual e-mail vai enviar a mensagem
        mimemsg['From']=mail_from
        # Seleciona o e-mail destino
        mimemsg['To']=mail_to
        # Define o Título do e-mail
        mimemsg['Subject']=mail_subject
        # Define a mensagem do e-mail
        mimemsg.attach(MIMEText(mail_body, 'plain'))

        # Enviar e-mail - Não alterar esta parte de código
        with open(mail_attachment, "rb") as attachment:
            mimefile = MIMEBase('application', 'octet-stream')
            mimefile.set_payload((attachment).read())
            encoders.encode_base64(mimefile)
            mimefile.add_header('Content-Disposition', "attachment; filename= %s" % mail_attachment_name)
            mimemsg.attach(mimefile)
            connection = smtplib.SMTP(host='smtp.gmail.com', port=587)
            connection.starttls()
            connection.login(username,password)
            connection.send_message(mimemsg)
            connection.quit()

# 4.2 - Enviar e-mails com registros inconsistentes ou incoompletos

In [115]:
# Lê o arquivo com as duplicidades a serem apagadas e aloca na variável hospitais
hospitais = pd.read_excel("bases/erros/todos_erros.xlsx")

# Transforma a variável hospitais em um array que contém os nomes dos hospitais presentes na planilha de duplicidades a serem apagadas
hospitais = hospitais.ID_UNIDADE.unique()

In [ ]:
# Cria uma variável rota com a rota onde a planilha será salva, caso deseje musar isto, alterar aqui
rota = 'bases/erros/erros por hospitais/'

# Cria uma variável extencao que serve para definir a extensão da planilha a ser salva
extencao = '.xlsx'

username = 'apcs.1502@gmail.com'
password = 'fomgnsjujrxcrfpc'
mail_from  = 'apcs.1502@gmail.com'

título_email = 'Registros duplicados - '
corpo_email = 'Segue em anexo os registros que identificamos com duplicados, por gentileza apague todos do sistema.'

# Cria uma lista com todos os e-mails presentes de hospitais na lista de e-mails - evita que de erro ao tentar enviar um e-mail para um hospital que não esteja na lista
lista_todos_emails = list()
for i in emails.keys():
    lista_todos_emails.append(i)


# Cria um loop que irá ocorrer a quantidade de hospitais na planilha df_repetidos_apagados_hospitais
for x in range(hospitais.shape[0]):
    # Cria uma variável nome_hospital com o nome do hospital que está sendo percorrido pela lista
    nome_hospital = hospitais[x]

    # Verifica se o nome do hospital tem e-mail
    if nome_hospital in lista_todos_emails:
    
        # Cria uma variável salvar que junta a rota, nome do hospital e a extensão Ex: bases/planilha dos hospitais apagar/HRAN.xlsx
        salvar = rota+nome_hospital+extencao

        # Seleciona pra qual e-mail a mensagem e a planilha serão enviados
        mail_to = emails[nome_hospital]
        # Título do e-mail
        mail_subject = título_email+nome_hospital
        # Corpo do e-mail
        mail_body = 'Favor Corrigir os registros que seguem em anexo nas planilhas.'

        # Arquivo a ser enviado (passa a rota do arquivo de cada hospital)
        mail_attachment=salvar
        # Nome e extensão do arquivo
        mail_attachment_name=nome_hospital+extencao

        mimemsg = MIMEMultipart()
        # seleciona qual e-mail vai enviar a mensagem
        mimemsg['From']=mail_from
        # Seleciona o e-mail destino
        mimemsg['To']=mail_to
        # Define o Título do e-mail
        mimemsg['Subject']=mail_subject
        # Define a mensagem do e-mail
        mimemsg.attach(MIMEText(mail_body, 'plain'))

        # Enviar e-mail - Não alterar esta parte de código
        with open(mail_attachment, "rb") as attachment:
            mimefile = MIMEBase('application', 'octet-stream')
            mimefile.set_payload((attachment).read())
            encoders.encode_base64(mimefile)
            mimefile.add_header('Content-Disposition', "attachment; filename= %s" % mail_attachment_name)
            mimemsg.attach(mimefile)
            connection = smtplib.SMTP(host='smtp.gmail.com', port=587)
            connection.starttls()
            connection.login(username,password)
            connection.send_message(mimemsg)
            connection.quit()